In [137]:
import requests
import json
from google.cloud import storage

database_id=""
secret_key=""
slack_token=""

def upload_gcloud_bucket(data_results):
    bucket_name = 'leetcode-notion-db'
    bucket = storage.Client().get_bucket(bucket_name)

    blob = bucket.blob('past_record.json')

    blob.upload_from_string(data=json.dumps(data_results,
                            ensure_ascii=False), content_type='application/json')
    return

def push_results_to_map(response,mapper):
    
    for num in response.json()["results"]:
        if "題號" not in num["properties"].keys():
            print(num["properties"])
            continue
        if num["properties"]["Person"]["people"]:

            question_num = num["properties"]["題號"]["number"]
            authors_json = num["properties"]["Person"]["people"]
            if mapper.get(question_num):
                continue
            authors_list = []
            for author in authors_json:
                if "name" not in author.keys():
                    continue
                authors_list.append(author["name"])
                if question_num in mapper.keys():
                    print(question_num)
            mapper[question_num] = authors_list

    return mapper
    
def crawl_notion_sorted():
    question_authors_map = {}
    header = {"Authorization": secret_key,"Notion-Version": "2021-05-13"}
    query = {
                "sorts": [{
                    "property": "題號",
                    "direction": "ascending"
                }]
            }
    response = requests.post(
            "https://api.notion.com/v1/databases/" + database_id + "/query", headers=header, json=query)
    
    push_results_to_map(response,question_authors_map)
    start_cursor = response.json()["next_cursor"]
    while start_cursor:
        query["start_cursor"]=start_cursor
        response = requests.post(
            "https://api.notion.com/v1/databases/" + database_id + "/query", headers=header, json=query)
        push_results_to_map(response,question_authors_map)
        start_cursor = response.json()["next_cursor"] 
        
    return question_authors_map
    
question_authors_map = crawl_notion_sorted()

# upload_gcloud_bucket(question_authors_map)
# with open("past_record.json","w") as fin:
#     json.dump(question_authors_map,fin)

print(len(question_authors_map))

434
